In [1]:
%load_ext autoreload
%autoreload
%matplotlib inline

from news_crawler import NewsCrawler
from IPython.display import display
from IPython.core.debugger import set_trace

# 0. 초기화

In [2]:
crawler = NewsCrawler(storage='local')

checking local storage... done
we have total 294727 articles (283303 downloaded, 11424 trashed)


# 1. URL을 모은다

In [3]:
# 제거고려: nextbigfuture, nbcnews, joongang, chosun

In [4]:
collect_result = crawler.collect(); display(*collect_result)

100.00% (313.80 seconds): Collecting URLs... theverge            

100.00% (0.74 seconds): Selecting URLs... cryptonews          

,collected,collected_unique,selected
all,25443,25376,3262
reuters,1114,1114,332
dailymail,2757,2757,315
euronews,1040,1040,245
investing.com,956,956,214
yahoo,375,375,154
businessinsider,570,570,141
usatoday,517,517,109
telegraph,813,813,107
rt,439,439,105


,url,pubs,actual_pub
0,https://aljazeera.com/news/2019/10/dozen-kille...,"aljazeera, axios",aljazeera
1,https://bbc.com/news/uk-50221856,"bbc, realclearpolitics",bbc
2,https://businessinsider.co.za/boris-johnson-cl...,"businessinsider, news24",businessinsider
3,https://foxnews.com/world/how-al-baghdadi-isis...,"fox, realclearpolitics",fox
4,https://hotair.com/archives/ed-morrissey/2019/...,"townhall, hotair",hotair
5,https://msnbc.com/msnbc/watch/as-trump-takes-a...,"msnbc, nbcnews",msnbc
6,https://nbcnews.com/better/lifestyle/here-s-ho...,"msnbc, nbcnews",nbcnews
7,https://nbcnews.com/better/lifestyle/owning-do...,"msnbc, nbcnews",nbcnews
8,https://nbcnews.com/business/autos/gm-toyota-f...,"msnbc, nbcnews",nbcnews
9,https://nbcnews.com/business/consumer/amazon-w...,"msnbc, nbcnews",nbcnews


# 2. 해당 URL의 뉴스를 다운로드한다

In [5]:
download_result = crawler.download(); download_result

1.13% (39.82 seconds): Downloading... bbc                 

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


40.16% (833.56 seconds): Downloading... cnbc                

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


100.00% (3012.50 seconds): Downloading... dailymail           

collect                             download          \
                     collected collected_unique selected downloaded trashed   
all                      25443            25376     3262       1929    1333   
reuters                   1114             1114      332        142     190   
dailymail                 2757             2757      315        255      60   
euronews                  1040             1040      245         17     228   
investing.com              956              956      214         75     139   
yahoo                      375              375      154        123      31   
businessinsider            570              570      141         99      42   
usatoday                   517              517      109         49      60   
telegraph                  813              813      107         98       9   
rt                         439              439      105         16      89   
metro                      245              245       99         86      13   
aljazeera                  479              479       96         16      80   
fox                        350              350       69         47      22   
news24                    1198             1197       67         65       2   
marketwatch                181              181       62         34      28   
axios                      126              122       55         41      14   
independent                727              727       52         52       0   
guardian                   143              143       48         36      12   
breitbart                  395              395       46         46       0   
cnn                        839              839       44         18      26   
washingtonpost             139              139       40         39       1   
bbc                        279              279       38         28      10   
thestreet                  129              129       36         15      21   
zdnet                      335              335       36         15      21   
arynews                    140              140       36         16      20   
realclearpolitics          155              136       34         21      13   
afr                        434              434       34         34       0   
indiatimes                 171              171       34         27       7   
abcnews                     98               98       33         33       0   
scmp                       330              330       30          1      29   
...                        ...              ...      ...        ...     ...   
theverge                  1183             1183       17         17       0   
msnbc                      136               98       15          3      12   
nbcnews                    198              197       15          0      15   
politico                   209              209       13         12       1   
cbc                        299              299       13         13       0   
vice                       104              104       12         12       0   
techcrunch                  20               20       11         11       0   
npr                        919              919       10          7       3   
nationalreview             721              719       10         10       0   
hotair                      85               85       10         10       0   
news.com.au                278              278        9          6       3   
nymag                      226              226        9          8       1   
blastingnews               120              120        9          2       7   
huffpost                    67               67        8          7       1   
cryptonews                 328              328        8          2       6   
cryptocoin                  44               44        7          7       0   
cointelegraph               71               71        6          6       0   
townhall                   613              612        6          5      

In [28]:
download_result.to_dict()

{('collect', 'collected'): {'all': 8645,
  'reuters': 764,
  'fox': 299,
  'investing.com': 510,
  'cnn': 623,
  'telegraph': 542,
  'news24': 270,
  'marketwatch': 137,
  'washingtonpost': 137,
  'wsj': 204,
  'aljazeera': 168,
  'thestreet': 128,
  'washingtontimes': 115,
  'nationalreview': 730,
  'guardian': 164,
  'financialpost': 113,
  'msnbc': 136,
  'nbcnews': 87,
  'zdnet': 239,
  'businessinsider': 76,
  'cnbc': 66,
  'bbc': 192,
  'afr': 86,
  'vice': 105,
  'techcrunch': 20,
  'axios': 21,
  'politico': 206,
  'realclearpolitics': 139,
  'nytimes': 99,
  'theverge': 947,
  'scmp': 252,
  'thehill': 88,
  'huffpost': 65,
  'chinadaily': 393,
  'morningstar': 32,
  'rt': 16,
  'indiatimes': 33,
  'cryptonews': 327,
  'time': 19,
  'japan-news': 70,
  'townhall': 0,
  'arirang': 0,
  'yahoo': 16,
  'americanconservative': 0,
  'atlantic': 0,
  'arynews': 0,
  'usatoday': 0,
  'thenextweb': 0,
  'nymag': 0,
  'thinkprogress': 0,
  'arstechnica': 0,
  'newrepublic': 0,
  'npr':

In [13]:
10000/50

200.0

Bigquery와 local 뉴스 갯수가 다를 것이다. Bigquery에 뉴스를 모두 업데이트 한 후 해당 repository에 문제가 생겨서, github에서 전일기준 데이터를 clone하여, 뉴스를 다시 다운로드 받은 날이 있다. 2019-10-22

In [15]:
275632+81

275713

In [18]:
from pathlib import Path
downloaded = [p.stem for p in Path('newsdata/downloaded').glob('**/*.json')]
trashed = [p.stem for p in Path('newsdata/trashed').glob('**/*.json')]

In [19]:
fnames_remove = set(downloaded) & set(trashed)

In [20]:
fnames_remove

{'364359fe8009b0eafc8c7698d136c85609507832',
 '48c17b8c7052dd982fec7893ad220f309311523e',
 '491d72f7e9c206d80afec8be4af247d2cee62b2d',
 '521cd5b023984b3dcd764aaf26e56a82e4bdf864',
 '6b3253676b0ac3b421e3b75138acf378edb867d4',
 '6d2fd1f6e66da4710074216d15f1f96fb1894355',
 'a02a1f32ccae0a118c17506ffad63bd7a31b84f3',
 'c82ed757f10fdfd53c28cb364e63bccf9947bef3',
 'f86363446716010bbad6138fe5ed767a36250441',
 'fce755ae86014651c8c3bf7061a7a0a6ca348460'}

In [21]:
for fname in fnames_remove:
    file = Path('newsdata/trashed/' + fname[:3] + '/' + fname + '.json')
    file.unlink()